In [ ]:
from model.calibration import fit_model, check_fit
from model.optimisation import optimise_interventions
from plotting.plots import plot_future_trajectories, make_intervention_piechart

minimised_outcomes = {
    "incidence_per100k": "TB incidence in 2040",
    "tb_deaths": "TB mortality in 2040",
    "cumulative_future_deaths": "TB deaths (2024-2040)",
    "cumulative_future_paed_deaths": "Paediatric TB deaths (2024-2040)",
    "years_of_life_lost": "Years of life lost (2025-2040)",
}


def run_analysis(target_incidence=100):

    bcm, mle_params = fit_model(target_incidence=target_incidence)
    check_fit(bcm, mle_params)
    derived_outputs = {"baseline": bcm.run(mle_params | {"decision_var_trans": 0., "decision_var_cdr": 0., "decision_var_pt": 0.}).derived_outputs}

    opti_decision_vars = {}
    for minimised_indicator in minimised_outcomes:
        opti_bcm, opti_params = optimise_interventions(mle_params, minimised_indicator=minimised_indicator)
        res = opti_bcm.run(opti_params)
        derived_outputs[minimised_indicator] = res.derived_outputs
        opti_decision_vars[minimised_indicator] = opti_params

    return derived_outputs, opti_decision_vars

In [ ]:
derived_outputs, opti_decision_vars = run_analysis(target_incidence=100)

In [ ]:
output_names = {
    "incidence_per100k": "TB incidence (/100k/y)",
    "ltbi_prevalence_perc": "LTBI prevalence (%)",
    "cumulative_future_deaths": "Cumulative TB deaths",
    "cumulative_future_paed_deaths": "Cumulative paediatric TB deaths",

}

from matplotlib import pyplot as plt


def plot_optimised_trajectories(derived_outputs, output="incidence_per100k", ax=None):

    if not ax:
        fig, ax = plt.subplots(1, 1)

    xmin = 2020
    ymax = 0.
    for sc_name, derived_df in derived_outputs.items():
        derived_df[output].loc[xmin:].plot(label=sc_name)
        ymax = max(ymax, derived_df[output].loc[xmin:].max())

    ax.set_ylabel(output_names[output])
    ax.set_ylim((0, 1.2 * ymax))
    ax.legend()


plot_optimised_trajectories(derived_outputs)

In [ ]:
plot_optimised_trajectories(derived_outputs, output="cumulative_future_paed_deaths")

In [ ]:
plot_optimised_trajectories(derived_outputs, output="cumulative_future_deaths")

In [ ]:
from plotting.plots import make_intervention_piechart
for sc_name, opti_vars in opti_decision_vars.items():
    ax = make_intervention_piechart(opti_vars)
    ax.set_title(sc_name)